# Solving Business Problems with AI

## Objective
Develop a proof of concept application that intelligently processes email order requests and customer inquiries for a fashion store. The system should categorize emails into product inquiries or order requests and generate appropriate responses based on product catalog information and stock status.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

## Instructions

- All requirements have to be implemented with the use of LLMs to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Because solving this assessment requires using advanced LLM capabilities, we provide you with a temporary OpenAI API key granting access to GPT-4o. You can use this key or your own, but please note that the provided key has a token quota, so use it wisely. We have carefully adjusted the limits to ensure they are sufficient for you to complete the task.
- Address the requirements in the order listed. Please review the requirements ahead to have a general implementation plan in place before you start.
- Your deliverables should include the code developed within this notebook, and a spreadsheet containing results across separate sheets. Comments detailing your thought process are appreciated.
- You are allowed to use additional libraries (langchain, etc.) in order to make solution more concise.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output:**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process order requests in the order they are received. For each request, verify product availability in stock. If the order can be fully fulfilled, create a new order line with the status **created**. If the order cannot be fully fulfilled, create a line with the status **out of stock**. After placing the order, update the stock to accurately reflect the current inventory levels.

    **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate and save response emails based on order processing results. Depending on the order status email should inform customer that their order was processed or could not be fulfilled. If the order was successfully processed, send an email to the customer informing them that their order has been processed, including details like the product name and quantity. If the order could not be fulfilled due to insufficient stock, send an email explaining the situation and specifying which items are out of stock. Optionally, offer options such as waiting for restock or choosing alternative products. Do your best to make an email look production ready!

    **Output**: Populate the **order-response** sheet with columns: email ID, response.

### 3. Handle product inquiry

Customers may ask general open questions.
- Respond to product inquiries using the information from the product catalog.
- Ensure that solution you provide scales well to handle the full catalog of over 100,000 products. This means that solutions which include raw product catalog in the prompt will be rated lower!

**Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria

- **Utilization of AI Tools**: Effectiveness in leveraging AI technologies provided (e.g., OpenAI API) to fulfill the task requirements. Employ your knowledge of various AI driven development techniques depending on the task.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational.
- **Code Quality and Clarity**: Code should be well-organized with clear logic and structure.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the spreadsheet. Double check before submitting!

We look forward to seeing your solution and how you approach solving real-world problems using AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [ ]:
key = 'a0BIj000001QtHNMA0'

In [ ]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key='<OPENAI API KEY: Use one provided by Crossover or your own>'
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

In [ ]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


First problem is simple classification. Asking chatgpt to do it shouldn't be a problem.

In [ ]:
# Install the OpenAI Python package.
%pip install openai

# Task 1. Classify emails

In [ ]:
from openai import OpenAI
import pandas as pd
from google.colab import files

# Set your OpenAI API key
client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key=key
)

# Example data
# data = {
#     'email_id': ['email_1', 'email_2', 'email_3'],
#     'email_text': [
#         "I would like to inquire about the availability of your new product line.",
#         "I want to place an order for 50 units of product X.",
#         "Can you provide more details on the specifications of product Y?"
#     ]
# }

# emails_df = pd.DataFrame(data)

def classify_email_with_chatgpt(email_text):
    # Send the email text to ChatGPT for classification
    response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
            {"role": "system", "content": "You are an assistant that classifies emails into 'product inquiry' or 'order request'. Do not say anything other than the classification text."},
            {"role": "user", "content": f"Classify this email: {email_text}"}
        ]
    )
    classification = str(response.choices[0].message.content).strip()
    return classification

# Apply the classification to each email
emails_df['category'] = emails_df['message'].apply(classify_email_with_chatgpt)

# Display the classified emails
# import ace_tools as tools; tools.display_dataframe_to_user(name="Classified Emails with ChatGPT", dataframe=emails_df)

emails_df.to_csv("email_categories.csv", index = False)
files.download("email_categories.csv")
emails_df[['email_id', 'category']].head(10)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,email_id,category
0,E001,order request
1,E002,product inquiry
2,E003,product inquiry
3,E004,order request
4,E005,product inquiry
5,E006,product inquiry
6,E007,order request
7,E008,order request
8,E009,product inquiry
9,E010,Order request


# Task 2. Process order requests

In [ ]:
emails_df = pd.read_csv("email_categories.csv")
emails_df.head(3)


,email_id,subject,message,category
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT...",order request
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V...",product inquiry
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and...",product inquiry


In [ ]:
import openai
import pandas as pd

# Set up your OpenAI API key
openai.api_key = 'your-openai-api-key'

# Sample data for products (replace this with your actual data)
# products_data = {
#     'product_id': ['prod_X', 'prod_Y'],
#     'name': ['Product X', 'Product Y'],
#     'stock': [100, 50]
# }
# products_df = pd.DataFrame(products_data)

# Example emails DataFrame with classifications
# emails_df = pd.DataFrame({
#     'email_id': ['email_1', 'email_2', 'email_3'],
#     'email_text': [
#         "I would like to inquire about the availability of your new product line.",
#         "I want to place an order for 50 units of Prodct X and 20 units of Product Y.",
#         "Can you provide more details on the specifications of product Y?"
#     ],
#     'category': ['product inquiry', 'order request', 'product inquiry']
# })

# Function to use ChatGPT to parse the email and extract valid product names and quantities
def extract_orders_with_chatgpt(email_text, valid_product_names):
    prompt = (
        "You are an AI that helps extract items and quantities from order emails. "
        "Given the email text below and the list of valid product names, "
        "please extract the valid product names and their corresponding quantities."
        "\n\nValid product names: " + ", ".join(valid_product_names) +
        "\n\nEmail: " + email_text +
        """\n\nOutput the result as a list of dictionaries with 'product_name' and 'quantity' keys. If request is for all available quantity of a product, quantity should be 'all'.
         If there is a range of numbers in quantity, quantity should be the biggest number If no products or quantities can be extracted, return an empty list. Give a crisp answer, with no extra text or explanations."""
    )

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    orders = eval(response.choices[0].message.content.strip())
    return orders

# Process each email
order_statuses = []
responses = []

def process_order(email_id, email_text):
    valid_product_names = products_df['name'].tolist()
    orders = extract_orders_with_chatgpt(email_text, valid_product_names)

    if not orders:
        order_statuses.append({
            'email_id': email_id,
            'product_id': None,
            'quantity': None,
            'status': 'invalid order'
        })
        responses.append({
            'email_id': email_id,
            'response': "Dear Customer, we couldn't process your order because the products or quantities specified are not valid."
        })
        return

    response_parts = []
    for order in orders:
        product_name = order['product_name'].strip()
        print(order['quantity'])
        quantity = int(order['quantity']) if not 'all' in str(order['quantity']) else list(products_df[products_df['name'] == product_name]['stock'])[0]

        # Find the corresponding product ID
        product_row = products_df[products_df['name'] == product_name].iloc[0]
        product_id = product_row['product_id']
        stock = product_row['stock']

        if stock >= quantity:
            status = 'created'
            products_df.loc[products_df['product_id'] == product_id, 'stock'] -= quantity
            response_parts.append(f"{quantity} units of {product_name} have been successfully processed.")
        else:
            status = 'out of stock'
            response_parts.append(f"Unfortunately, we do not have sufficient units of {product_name} available, so your order for {quantity} units could not be fully processed.")

        order_statuses.append({
            'email_id': email_id,
            'product_id': product_id,
            'quantity': quantity,
            'status': status
        })

    response = "Dear Customer, " + " ".join(response_parts)
    responses.append({
        'email_id': email_id,
        'response': response
    })

# Run the processing for each email classified as an "order request"
for _, row in emails_df.iterrows():
    if row['category'] == 'order request':
        process_order(row['email_id'], row['message'])

# Convert the order statuses and responses to DataFrames
order_status_df = pd.DataFrame(order_statuses)
order_response_df = pd.DataFrame(responses)

# Display the results
# import ace_tools as tools; tools.display_dataframe_to_user(name="Order Status", dataframe=order_status_df)
# tools.display_dataframe_to_user(name="Order Responses", dataframe=order_response_df)

order_status_df, order_response_df
order_status_df.to_csv("order_status.csv", index = False)
files.download("order_status.csv")
order_response_df.to_csv("order_response.csv", index = False)
files.download("order_response.csv")


all
4
5
2
1
1
1
1


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task 3. Handle product inquiry

In [ ]:
%pip install sentence_transformers
%pip install faiss-cpu

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load the product catalog (assumed to be in products_df)
# products_df = pd.DataFrame({
#     'product_id': ['prod_X', 'prod_Y'],
#     'name': ['Product X', 'Product Y'],
#     'description': [
#         'Product X is a high-quality widget designed for durability.',
#         'Product Y is an advanced gadget with cutting-edge technology.'
#     ]
# })

# Initialize the model for generating embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for product descriptions
descriptions = products_df['description'].tolist()
embeddings = model.encode(descriptions)

# Convert embeddings to a format suitable for FAISS
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)  # L2 distance

# Add the embeddings to the index
index.add(np.array(embeddings).astype('float32'))

# Save product information along with embeddings for later retrieval
product_info = products_df[['product_id', 'name', 'description']].copy()


In [ ]:
# import openai

# Set your OpenAI API key
# openai.api_key = 'your-openai-api-key'

# # Example emails DataFrame with classifications
# emails_df = pd.DataFrame({
#     'email_id': ['email_1', 'email_2', 'email_3'],
#     'email_text': [
#         "Can you tell me more about your high-quality widgets?",
#         "I would like to inquire about your advanced gadgets.",
#         "What are the features of Product X?"
#     ],
#     'category':['product inquiry', 'product inquiry', 'product inquiry']
# })

# # Classify emails as product inquiries
# def classify_email(email_text):
#     response = client.chat.completions.create(
#         model="gpt-4",
#         messages=[
#             {"role": "system", "content": "You are an AI that classifies emails into 'product inquiry' or 'other'."},
#             {"role": "user", "content": f"Classify this email: {email_text}"}
#         ]
#     )
#     category = response.choices[0].message.content.strip()
#     return category

# emails_df['category'] = emails_df['email_text'].apply(classify_email)


In [ ]:
def generate_response(email_text, retrieved_description):
    prompt = (
        f"You are an AI assistant. A customer has inquired about a product. "
        f"Using the following product description, respond to the inquiry in a friendly manner."
        f"\n\nProduct Description: {retrieved_description}"
        f"\n\nCustomer Inquiry: {email_text}"
        f"\n\nPlease generate a helpful response."
    )

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def process_product_inquiry(email_id, email_text):
    # Generate an embedding for the email
    email_embedding = model.encode([email_text])

    # Search for the most similar product description
    D, I = index.search(np.array(email_embedding).astype('float32'), k=1)

    # Retrieve the most relevant product description
    best_match_index = I[0][0]
    best_match_description = "product id: " + product_info.iloc[best_match_index]['product_id'] + ", product name: " + product_info.iloc[best_match_index]['name'] + ", description: " + product_info.iloc[best_match_index]['description']

    # Generate a response based on the best match
    response_text = generate_response(email_text, best_match_description)

    return response_text

# Process emails classified as "product inquiry"
responses = []
for _, row in emails_df.iterrows():
    if row['category'] == 'product inquiry':
        response = process_product_inquiry(row['email_id'], row['message'])
        responses.append({'email_id': row['email_id'], 'response': response})

# Convert responses to a DataFrame for output
response_df = pd.DataFrame(responses)

# Display the results
# import ace_tools as tools; tools.display_dataframe_to_user(name="Product Inquiry Responses", dataframe=response_df)

print(response_df.head(10))

response_df.to_csv("inquiry_response.csv", index = False)
files.download("inquiry_response.csv")


  email_id                                           response
0     E002  Good morning Jessica! \n\nIt sounds like you h...
1     E003  Hello David,\n\nThanks for your interest in ou...
2     E005  Good day! \n\nThank you for your interest in o...
3     E006  Hello Sam,\n\nThank you for reaching out and s...
4     E009  Hola, gracias por su pregunta. Parece que hay ...
5     E011  Hello!\n\nThank you for reaching out about our...
6     E015  Good morning! What a lovely gift idea for your...
7     E016  Hello Claire,\n\nThank you for reaching out! O...
8     E020  Hello Antonio,\n\nThanks for your interest in ...
9     E021  Hello,\n\nThank you for reaching out and for y...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>